In [3]:
# import packages

import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats import proportion as proptests

import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
# import data

data = pd.read_csv('../data/statistical_significance_data.csv')
data.head(10)

,condition,click
0,1,0
1,0,0
2,0,0
3,1,1
4,1,0
5,1,0
6,0,0
7,1,1
8,0,0
9,1,0


### Checking the Invariant Metric
1. Check if the number of visitors assigned to each group is similar.
2. If two groups are imbalanced on the invariant metric, how the visitors were split 
so that any sources of bias are accounted for
3. Two-sided hypothesis test on the proportion of visitors assigned to one of our conditions.
4. Simulation-based approach: Simulate the number of visitors that would be assigned to each group
for the number of total observations.
5. Analytic approach -> Use binomial distribution to compute a p-value for the test.

### Analytic Approach

In [12]:
# get number of trials and number of 'successes'
n_obs = data.shape[0]
n_control = data.groupby('condition').size()[0]

In [13]:
# compute z-score and p-value
p = 0.5
sd = np.sqrt(p * (1-p) * n_obs)
z = ((n_control + .5) - p * n_obs) / sd

print(z)
print(2 * stats.norm.cdf(z))

-0.5062175977346661
0.6127039025537114


### Simulation Approach

In [14]:
# simulate outcomes under null, compare to observed outcome
p = 0.5
n_trials = 200000

samples = np.random.binomial(n_obs, p, n_trials)

print(np.logical_or(samples <= n_control, samples >= (n_obs - n_control)).mean())

0.61198


### Checking the Evaluation Metric


In [15]:
p_click = data.groupby('condition').mean()['click']
p_click

condition
0    0.079430
1    0.112205
Name: click, dtype: float64

In [16]:
p_click[1] - p_click[0]

0.03277498917523293

### Analytic Approach

In [17]:
# get number of trials and overall 'success' rate under null
n_exper = data.groupby('condition').size()[1]
p_null = data['click'].mean()

In [18]:
#compute standard error, z-score, and p-value
se_p = np.sqrt(p_null * (1-p_null) * (1/n_control + 1/n_exper))

z = (p_click[1] - p_click[0]) / se_p

print(z)
print(1-stats.norm.cdf(z))

1.7571887396196666
0.039442821974613684


### Simulation Approach 

In [20]:
# simulate outcomes under null, compare to observed outcome
n_trials = 200000

ctrl_clicks = np.random.binomial(n_control, p_null, n_trials)
exp_clicks = np.random.binomial(n_exper, p_null, n_trials)
samples = exp_clicks / n_exper - ctrl_clicks / n_control

print((samples >= (p_click[1] - p_click[0])).mean())

0.039795
